In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%matplotlib inline

In [0]:
import torch
print(torch.__version__)

1.1.0


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
import torch.nn.init as init
print(torch.__version__)
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import pickle


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

1.1.0


In [0]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self):        
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [0]:
MAX_LENGTH = 10  # Maximum sentence length to consider

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile):
    print("Reading lines...")    
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc()
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(datafile):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs

datafile = '/content/drive/My Drive/Deep Learning/Assignment 5/chatDataset.txt'
voc, total_pairs = loadPrepareData(datafile)
# Print some pairs to validate
print("\npairs:")
for pair in total_pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [0]:
# split pairs into train and test data here, use test data to compute bleu score later on
print(len(total_pairs))
test_pairs =total_pairs[60000:]
pairs = total_pairs[:59999]
print(len(test_pairs))

64271
4271


In [0]:
#input_variable

MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345
Trimmed from 59999 pairs to 49638, 0.8273 of total


In [0]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches
max_target_len
input_variable

tensor([[  65,  716,   51, 4692,  318],
        [2559,  119,  109,    6,    6],
        [   7,   22, 1264,    2,    2],
        [ 144,    6,    4,    0,    0],
        [  53,    2,    2,    0,    0],
        [7220,    0,    0,    0,    0],
        [   6,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        #init.normal_(self.embedding.weight, 0.0, 0.2)
        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.LSTM = nn.LSTM(hidden_size,hidden_size,num_layers=n_layers,dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

        
    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embeddings = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        #pack_padded_seq = torch.nn.utils.rnn.pack_padded_sequence(embeddings, input_lengths)
        # Forward pass through GRU
        LSTM_output, LSTM_hidden = self.LSTM(embeddings, hidden)
        # Unpack padding
        #unpack_padded_seq, _ = torch.nn.utils.rnn.pad_packed_sequence(LSTM_output)
        # Sum bidirectional GRU outputs
        #outputs = unpack_padded_seq[:, :, :self.hidden_size] + unpack_padded_seq[:, : ,self.hidden_size:]
        outputs = LSTM_output[:, :, :self.hidden_size] + LSTM_output[:, : ,self.hidden_size:]
        # Return output and final hidden state
        hidden = LSTM_hidden
        return outputs, hidden

In [0]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [0]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers such as embedding, dropout, GRU etc.
        self.embedding = embedding
        #init.normal_(self.embedding.weight, 0.0, 0.2)
        self.dropout = nn.Dropout(dropout)
        self.LSTM = nn.LSTM(hidden_size, hidden_size, num_layers=n_layers, dropout=(0 if n_layers == 1 else dropout))#uni-directional GRU
        print(self.LSTM)
        self.fc1 = nn.Linear(hidden_size * 2, hidden_size) #from above attention module 
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedding = self.embedding(input_step)
        embedding = self.dropout(embedding) 
        
        # Forward through unidirectional GRU
        LSTM_output, LSTM_hidden = self.LSTM(embedding, last_hidden)
        # Calculate attention weights from the current GRU output
        weights = self.attn(LSTM_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context_vector = torch.bmm(weights,encoder_outputs.transpose(0, 1)) 
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        LSTM_output = LSTM_output.squeeze(0) 
        context_vector = context_vector.squeeze(1) 
        concatenated = torch.cat((LSTM_output, context_vector), 1)
        fc1output = self.fc1(concatenated) #pass through first fc layer
        fc1output = torch.tanh(fc1output) 
        fc2output = self.fc2(fc1output)
        fc2output = torch.tanh(fc2output)
        output = self.fc3(fc2output)
        output = F.softmax(output, dim=1)
        # Predict next word using Luong eq. 6
        # Return output and final hidden state
        hidden = LSTM_hidden
        return output, hidden

In [0]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

Define Evaluation
-----------------

After training a model, we want to be able to talk to the bot ourselves.
First, we must define how we want the model to decode the encoded input.

Greedy decoding
~~~~~~~~~~~~~~~

Greedy decoding is the decoding method that we use during training when
we are **NOT** using teacher forcing. In other words, for each time
step, we simply choose the word from ``decoder_output`` with the highest
softmax value. This decoding method is optimal on a single time-step
level.

To facilite the greedy decoding operation, we define a
``GreedySearchDecoder`` class. When run, an object of this class takes
an input sequence (``input_seq``) of shape *(input_seq length, 1)*, a
scalar input length (``input_length``) tensor, and a ``max_length`` to
bound the response sentence length. The input sentence is evaluated
using the following computational graph:

**Computation Graph:**

   1) Forward input through encoder model.
   2) Prepare encoder's final hidden layer to be first hidden input to the decoder.
   3) Initialize decoder's first input as SOS_token.
   4) Initialize tensors to append decoded words to.
   5) Iteratively decode one word token at a time:
       a) Forward pass through decoder.
       b) Obtain most likely word token and its softmax score.
       c) Record token and score.
       d) Prepare current token to be next decoder input.
   6) Return collections of word tokens and scores.




In [0]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        #encoder_h = torch.zeros(encoder.n_layers*2, batch_size, encoder.hidden_size)
        #encoder_h = encoder_h.to(device)
        #encoder_c = torch.zeros(encoder.n_layers*2, batch_size, encoder.hidden_size)
        #encoder_c = encoder_c.to(device)
        encoder_output, encoder_hidden = encoder(input_seq, input_length)
        encoder_h, encoder_c = encoder_hidden

        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        #decoder_hidden = e_hidden[:encoder.n_layers]
        decoder_h = encoder_h[:encoder.n_layers]
        decoder_c = encoder_c[:encoder.n_layers]
     
        # Initialize decoder input with SOS_token
        decoder_input = torch.LongTensor([[SOS_token]])
        decoder_input = decoder_input.to(device) # to gpu, if available
        # Initialize tensors to append decoded words to
        decoded_output = torch.zeros([0], device=device, dtype=torch.long)
        score = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, (decoder_h,decoder_c), encoder_output)
            # Obtain most likely word token and its softmax score
            softmax_score, word_token = torch.max(decoder_output, dim=1)
            # Record token and score
            decoded_output = torch.cat((decoded_output, word_token), dim=0)
            score = torch.cat((score, softmax_score), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(word_token, 0)
        # Return collections of word tokens and scores
        all_tokens = decoded_output
        all_scores = score
        return all_tokens, all_scores

Evaluate my text
----

Now that we have our decoding method defined, we can write functions for
evaluating a string input sentence. The ``evaluate`` function manages
the low-level process of handling the input sentence. We first format
the sentence as an input batch of word indexes with *batch_size==1*. We
do this by converting the words of the sentence to their corresponding
indexes, and transposing the dimensions to prepare the tensor for our
models. We also create a ``lengths`` tensor which contains the length of
our input sentence. In this case, ``lengths`` is scalar because we are
only evaluating one sentence at a time (batch_size==1). Next, we obtain
the decoded response sentence tensor using our ``GreedySearchDecoder``
object (``searcher``). Finally, we convert the response’s indexes to
words and return the list of decoded words.

``evaluateInput`` acts as the user interface for our chatbot. When
called, an input text field will spawn in which we can enter our query
sentence. After typing our input sentence and pressing *Enter*, our text
is normalized in the same way as our training data, and is ultimately
fed to the ``evaluate`` function to obtain a decoded output sentence. We
loop this process, so we can keep chatting with our bot until we enter
either “q” or “quit”.

Finally, if a sentence is entered that contains a word that is not in
the vocabulary, we handle this gracefully by printing an error message
and prompting the user to enter another sentence.




In [0]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    words_to_indexes = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(i) for i in words_to_indexes])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(words_to_indexes).transpose(0, 1)
    #print(input_batch)
    #print(lengths)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence by removing EOS tokens
            answer = []
            for word in output:
                if (word == 'EOS') or (word == 'PAD'):
                  continue
                else:
                    answer.append(word)
            print('answer:')
            print(' '.join(answer))

        except KeyError:
            print("Error: Encountered unknown word.")

Run Model
---------

Finally, it is time to run our model!

Regardless of whether we want to train or test the chatbot model, we
must initialize the individual encoder and decoder models. In the
following block, we set our desired configurations, choose to start from
scratch or set a checkpoint to load from, and build and initialize the
models. Feel free to play with different model configurations to
optimize performance.

In [0]:
# Configure models
model_name = 'LSTM_greedy_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 5000
print_every = 200
save_every = 20

file = '/content/drive/My Drive/Deep Learning/Assignment 5/5000_LSTM_greedy_model.tar'
model = torch.load(file)
encoder_ = model['encoder']
encoder_optimizer_ = model['encoder_optimizer']
decoder_ = model['decoder']
decoder_optimizer_ = model['decoder_optimizer']
embeddings = model['embedding']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
embedding.load_state_dict(embeddings)

# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
encoder.load_state_dict(encoder_)
decoder.load_state_dict(decoder_)

# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

encoder_optimizer.load_state_dict(encoder_optimizer_)
decoder_optimizer.load_state_dict(decoder_optimizer_)

Building encoder and decoder ...
LSTM(500, 500, num_layers=2, dropout=0.1)
Models built and ready to go!
Building optimizers ...


In [0]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

In [0]:
# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

> can t you go to sleep ?
answer:
no . . . . .
> why not
answer:
i don t . . . .
> you should
answer:
i don t . . . .
> what is your name
answer:
i m going to the window . .
> which window ?
answer:
the current . . . .


KeyboardInterrupt: ignored

In [0]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import numpy as np

def BLEU_SCORE(encoder, decoder, searcher, voc, pairs):
    print(len(pairs))
    bleu_scores = []
    candidates = []
    #small_batch_size = 5
    #batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
    #input_variable, lengths, target_variable, mask, max_target_len = batches
    for i in range(len(pairs)):
      try:
            #print(i)
            Question = pairs[i][0]
            reference_answer = pairs[i][1]
            candidate_answer = evaluate(encoder, decoder, searcher, voc, Question)
            #print(Question)
            #print(reference_answer)
            #print(candidate_answer)
            candidate = []
            for word in candidate_answer:
                if (word == 'EOS') or (word == 'PAD'):
                  continue
                else:
                    candidate.append(word)
            
            #print(candidate)
            candidates.append(candidate)
            bleu_score = sentence_bleu(reference_answer, candidate, weights=(1, 0, 0, 0), smoothing_function=SmoothingFunction().method1)
            bleu_scores.append(bleu_score)
            #print(bleu_scores)
            #break
            
      except KeyError:
            #print(i)
            #print("Error: Encountered unknown word.")
            #bleu_scores[i] = 0.0
            continue
    return bleu_scores, candidates

In [0]:
scores, generated_answers = BLEU_SCORE(encoder, decoder, searcher, voc, test_pairs)

4271


In [0]:
print(len(scores))
print(np.mean(scores))
print(np.max(scores))

3832
0.1962144182655665
1.0
